In [ ]:
import numpy as np
from coffeebrain import dataset
from coffeebrain import constants
from cStringIO import StringIO
from sklearn.svm import SVC
from PIL import Image
import IPython.display

def feature_vector_to_image(a, fmt='png'):
    """
    Helper method for displaying images inside the jupyter playbook
    """
    a = a.reshape((100,100)) 
    a = np.uint8(a)
    f = StringIO()
    Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

# Load the datasets for both sides
# dataloader.py should be called beforehand
print "loading left datasets"
left_training_labels, left_training_features = dataset.load_dataset_from_disk('left', 'training')
left_testing_labels, left_testing_features = dataset.load_dataset_from_disk('left', 'testing')

print "loading right datasets"
right_training_labels, right_training_features = dataset.load_dataset_from_disk('right', 'training')
right_testing_labels, right_testing_features = dataset.load_dataset_from_disk('right', 'testing')

# Train left classifier
print "training left"
left_classifier = SVC(kernel="linear", probability=True)
left_classifier.fit(left_training_features, left_training_labels)

# Train right classifier
print "training right"
right_classifier = SVC(kernel="linear", probability=True)
right_classifier.fit(right_training_features, right_training_labels)

# Score the classifiers
left_accuracy = left_classifier.score(left_testing_features, left_testing_labels)
right_accuracy = right_classifier.score(right_testing_features, right_testing_labels)

print "left accuracy was %.2f percent" % (left_accuracy *100)
print "right accuracy was %.2f percent" % (right_accuracy *100)

print "\n\nLEFT PREDICTIONS THAT WERE WRONG"

for i, label in enumerate(left_testing_labels):
    testing_label = left_testing_labels[i]
    prediction_result = left_classifier.predict([left_testing_features[i]])[0]
    prediction_probablity = left_classifier.predict_proba([left_testing_features[i]])[0].max()
    if prediction_result != testing_label:
        print "predicted %s (but should be %s) with confidence %.2f" % (constants.REVERSE_LABELS[prediction_result], constants.REVERSE_LABELS[testing_label], prediction_probablity *100)
        feature_vector_to_image(left_testing_features[i])

print "\n\nRIGHT PREDICTIONS THAT WERE WRONG"
for i, label in enumerate(right_testing_labels):
    testing_label = right_testing_labels[i]
    prediction_result = right_classifier.predict([right_testing_features[i]])[0]
    prediction_probablity = right_classifier.predict_proba([right_testing_features[i]])[0].max()
    if prediction_result != testing_label:
        print "predicted %s (but should be %s) with confidence %.2f" % (constants.REVERSE_LABELS[prediction_result], constants.REVERSE_LABELS[testing_label], prediction_probablity *100)
        feature_vector_to_image(right_testing_features[i])